![SETI](https://earthsky.org/upl/2020/02/Earth-transit-zone-Breakthrough-Listen.jpg)
# About This Notebook
This is a first run through the competition data to try and understand the datatset and realise the problem at hand with some quick EDA and a baseline Model.  
**If you found this notebook useful and use parts of it in your work, please don't forget to show your appreciation by upvoting this kernel. That keeps me motivated and inspires me to write and share these public kernels.** 😊

# Problem Statement
* The Breakthrough Listen team at the University of California, Berkeley, employs the world’s most powerful telescopes to scan millions of stars for signs of technology.
* It’s hard to search for a faint needle of alien transmission in the huge haystack of detections from modern technology.
* Current methods use two filters to search through the haystack.
    * First, the Listen team intersperses scans of the target stars with scans of other regions of sky. Any signal that appears in both sets of scans probably isn’t coming from the direction of the target star.
    * Second, the pipeline discards signals that don’t change their frequency, because this means that they are probably nearby the telescope.
* Use data science skills to help identify anomalous signals in scans of Breakthrough Listen targets.
* Because there are no confirmed examples of alien signals to use to train machine learning algorithms, the team included some simulated signals.

# Why this competition?
As evident from the problem statement, this competition presents an interesting challenge straight out of a Sci-Fi movie stuff!  
Also (if successful) this model should be able to answer one of the biggest questions in science.

# Expected Outcome
Given a numpy array of signal, we should be able to identify it as a positive class (signal from an alien lifeform) or negative class (signal from one of our devices).

# Data Description
Data is stored in a numpy float16 format in training folder and the labes are mentioned in the `train_labels.csv` file where the first letter of the file name indicates the subfolder the `.npy` file is placed inside the train directory.  
The data consist of two-dimensional arrays {shape = (6, 273, 256)}, so there may be approaches from computer vision that are promising, as well as digital signal processing, anomaly detection, and more.

# Grading Metric
Submissions are evaluated on **area under the ROC curve** between the predicted probability and the observed target.

# Problem Category
From the data and objective its is evident that this is a **Classification Problem**. But we have an option for the approach starting with vanilla ML methods to Computer Vision to Anomaly detection etc.

So without further ado, let's now start with some basic imports to take us through this:-

In [ ]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

In [ ]:
# Asthetics
import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# General
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import numpy as np
import os
import random
import glob
pd.set_option('display.max_columns', None)

# Visualizations
from PIL import Image
from plotly.subplots import make_subplots
from plotly.offline import iplot
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px
%matplotlib inline
sns.set(style="whitegrid")

# Image Aug
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

# Machine Learning
# Utils
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# Deep Learning
import torch
import torchvision
import timm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingLR
#Metrics
from sklearn.metrics import roc_auc_score

# Random Seed Initialize
RANDOM_SEED = 42

def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything()

# Device Optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')

In [ ]:
csv_dir = '../input/seti-breakthrough-listen'
train_dir = '../input/seti-breakthrough-listen/train'
test_dir = '../input/seti-breakthrough-listen/test'

train_file_path = os.path.join(csv_dir, 'train_labels.csv')
sample_sub_file_path = os.path.join(csv_dir, 'sample_submission.csv')

print(f'Train file: {train_file_path}')
print(f'Train file: {sample_sub_file_path}')

# EDA

In [ ]:
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(sample_sub_file_path)

In [ ]:
train_df.sample(10)

In [ ]:
test_df.sample(10)

Let's observe the class imbalance in the target variable...

In [ ]:
ax = plt.subplots(figsize=(12, 6))
sns.set_style("whitegrid")
sns.countplot(x='target', data=train_df);
plt.ylabel("No. of Observations", size=20);
plt.xlabel("Target", size=20);

As we can see (and sort of also hoped for), this is a pretty imbalanced dataset. We need to take care of that while building our models...

In [ ]:
def return_filpath(name, folder=train_dir):
    path = os.path.join(folder, name[0], f'{name}.npy')
    return path

In [ ]:
# Inspired from https://www.kaggle.com/ihelon/signal-search-exploratory-data-analysis
def show_cadence(filename, label=None, folder=train_dir):
    plt.figure(figsize=(16, 10))
    arr = np.load(return_filpath(filename, folder=folder))
    for i in range(6):
        plt.subplot(6, 1, i + 1)
        if i == 0:
            if label is not None:
                plt.title(f'ID: {os.path.basename(filename)} TARGET: {label}', fontsize=18)
        plt.imshow(arr[i].astype(float), interpolation='nearest', aspect='auto')
        plt.text(5, 100, ['ON', 'OFF'][i % 2], bbox={'facecolor': 'white'})
        plt.xticks([])
    plt.show()

In [ ]:
# Inspired from https://www.kaggle.com/ihelon/signal-search-exploratory-data-analysis
def show_channels(filename, label=None, folder=train_dir):
    plt.figure(figsize=(16, 10))
    if label is not None:
        plt.suptitle(f'ID: {os.path.basename(filename)} TARGET: {label}', fontsize=18)
    arr = np.load(return_filpath(filename, folder=folder))
    for i in range(6):
        plt.subplot(2, 3, i + 1)
        plt.imshow(arr[i].astype(float))
        plt.grid(b=None)
    plt.show()

Let's see some examples of the negative class:-

In [ ]:
df_tmp = train_df[train_df['target'] == 0].sample(3)
for _, row in df_tmp.iterrows():
    show_cadence(filename = row['id'],
                 label = row['target'])

In [ ]:
for _, row in df_tmp.iterrows():
    show_channels(filename = row['id'],
                 label = row['target'])

Now looking at some data of positive class:-

In [ ]:
df_tmp = train_df[train_df['target'] == 1].sample(3)
for _, row in df_tmp.iterrows():
    show_cadence(filename = row['id'],
                 label = row['target'])

In [ ]:
for _, row in df_tmp.iterrows():
    show_channels(filename = row['id'],
                 label = row['target'])

# Model

## Image Augmentation

In [ ]:
def get_train_transforms():
    return albumentations.Compose(
        [
            albumentations.Resize(256,256),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.Rotate(limit=180, p=0.7),
            albumentations.RandomBrightness(limit=0.6, p=0.5),
            albumentations.Cutout(
                num_holes=10, max_h_size=12, max_w_size=12,
                fill_value=0, always_apply=False, p=0.5
            ),
            albumentations.ShiftScaleRotate(
                shift_limit=0.25, scale_limit=0.1, rotate_limit=0
            ),
            ToTensorV2(p=1.0),
        ]
    )

def get_valid_transforms():
    return albumentations.Compose(
        [
            albumentations.Resize(256,256),
            ToTensorV2(p=1.0)
        ]
    )

def get_test_transforms(TTA):
    if TTA > 1:
        return albumentations.Compose(
            [
                albumentations.Resize(256,256),
                albumentations.HorizontalFlip(p=0.5),
                albumentations.VerticalFlip(p=0.5),
                albumentations.Rotate(limit=180, p=0.7),
                albumentations.RandomBrightness(limit=0.6, p=0.5),
                albumentations.ShiftScaleRotate(
                    shift_limit=0.25, scale_limit=0.1, rotate_limit=0
                ),
                ToTensorV2(p=1.0)
            ]
        )
    else:
        return albumentations.Compose(
            [
                albumentations.Resize(256,256),
                ToTensorV2(p=1.0)
            ]
        )

## Dataset

In [ ]:
train_df['image_path'] = train_df['id'].apply(lambda x: return_filpath(x))
test_df['image_path'] = test_df['id'].apply(lambda x: return_filpath(x, folder=test_dir))

In [ ]:
(X_train, X_valid, y_train, y_valid) = train_test_split(train_df['image_path'],
                                                        train_df['target'],
                                                        test_size=0.2,
                                                        stratify=train_df['target'],
                                                        shuffle=True,
                                                        random_state=RANDOM_SEED)

In [ ]:
class SETIDataset(Dataset):
    def __init__(self, images_filepaths, targets, transform=None):
        self.images_filepaths = images_filepaths
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = np.load(image_filepath)
        image = image.astype(np.float32)
        image = np.vstack(image).transpose((1, 0))
            
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        else:
            image = image[np.newaxis,:,:]
            image = torch.from_numpy(image).float()
        
        label = torch.tensor(self.targets[idx]).float()
        return image, label

In [ ]:
train_dataset = SETIDataset(
    images_filepaths=X_train.values,
    targets=y_train.values,
    transform=get_train_transforms()
)

valid_dataset = SETIDataset(
    images_filepaths=X_valid.values,
    targets=y_valid.values,
    transform=get_valid_transforms()
)

## Metrics

In [ ]:
def usr_roc_score(output, target):
    try:
        y_pred = torch.sigmoid(output).cpu()
        y_pred = y_pred.detach().numpy()
        target = target.cpu()

        return roc_auc_score(target, y_pred)
    except:
        return 0.5

In [ ]:
class MetricMonitor:
    def __init__(self, float_precision=3):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"],
                    float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )

## Scheduler

In [ ]:
scheduler_params = {
    'name': 'CosineAnnealingLR',
    'T_max': 10,
    'min_lr': 1e-6,
}

In [ ]:
def get_scheduler(optimizer, scheduler_params=scheduler_params):
    scheduler = CosineAnnealingLR(optimizer,
                                  T_max=scheduler_params['T_max'],
                                  eta_min=scheduler_params['min_lr'],
                                  last_epoch=-1)
    return scheduler

## Data Sampler

In [ ]:
class_counts = y_train.value_counts().to_list()
num_samples = sum(class_counts)
labels = y_train.to_list()

class_weights = [num_samples/class_counts[i] for i in range(len(class_counts))]
weights = [class_weights[labels[i]] for i in range(int(num_samples))]
sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))

## Loss Functions

### 1. Focal Cosine Loss

In [ ]:
focal_loss_params = {
    'alpha': 1,
    'gamma': 2,
    'xent': 0.1,
    'device': device
}

In [ ]:
class FocalCosineLoss(nn.Module):
    def __init__(self, focal_loss_params=focal_loss_params):
        super(FocalCosineLoss, self).__init__()
        self.alpha = focal_loss_params['alpha']
        self.gamma = focal_loss_params['gamma']
        self.xent = focal_loss_params['xent']
        if focal_loss_params['device'] == 'cuda':
            self.y = torch.Tensor([1]).cuda()
        else:
            self.y = torch.Tensor([1]).cpu()

    def forward(self, input, target, reduction='mean'):
        cosine_loss = F.cosine_embedding_loss(input, F.one_hot(target, num_classes=input.size(-1)), self.y, reduction=reduction)
        cent_loss = F.cross_entropy(F.normalize(input), target, reduce=False)
        pt = torch.exp(-cent_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * cent_loss

        if reduction == 'mean':
            focal_loss = torch.mean(focal_loss)

        return cosine_loss + self.xent * focal_loss

### 2. Label Smoothing

In [ ]:
label_smoothing_params = {
    'smoothing': 1,
    'classes': 2
}

In [ ]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes=label_smoothing_params['classes'],
                 smoothing=label_smoothing_params['smoothing']):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
    
    def forward(self, pred, target):
        pred = pred.log_softmax(dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

## CNN Model

In [ ]:
params = {
    'model': 'nfnet_l0',
    'inp_channels': 1,
    'device': device,
    'lr': 1e-4,
    'weight_decay': 1e-6,
    'batch_size': 32,
    'num_workers' : 0,
    'epochs': 10,
    'out_features': 1,
    'Balance_Dataset': True
}

In [ ]:
if params['Balance_Dataset']:
    train_loader = DataLoader(
        train_dataset, batch_size=params['batch_size'], sampler = sampler,
        num_workers=params['num_workers'], pin_memory=True
    )
else:
    train_loader = DataLoader(
        train_dataset, batch_size=params['batch_size'], shuffle=True,
        num_workers=params['num_workers'], pin_memory=True
    )

val_loader = DataLoader(
    valid_dataset, batch_size=params['batch_size'], shuffle=False,
    num_workers=params['num_workers'], pin_memory=True
)

In [ ]:
class AlienNet(nn.Module):
    def __init__(self, model_name=params['model'], out_features=params['out_features'],
                 inp_channels=params['inp_channels'], pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained,
                                       in_chans=inp_channels)
        if model_name.split('_')[0] == 'efficientnet':
            n_features = self.model.classifier.in_features
            self.model.conv_stem = nn.Conv2d(inp_channels, 40, kernel_size=(3, 3),
                                             stride=(2, 2), padding=(1, 1), bias=False)
            self.model.classifier = nn.Linear(n_features, out_features)
        
        elif model_name.split('_')[0] == 'nfnet':
            n_features = self.model.head.fc.in_features
            self.model.head.fc = nn.Linear(n_features, out_features)
    
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
model = AlienNet()
model = model.to(params['device'])
criterion = nn.BCEWithLogitsLoss().to(params['device'])
optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'],
                             weight_decay=params['weight_decay'],
                             amsgrad=False)
scheduler = get_scheduler(optimizer)

## Training and Validation

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch, params):
    metric_monitor = MetricMonitor()
    model.train()
    stream = tqdm(train_loader)
    max_grad_norm = 1000
    for i, (images, target) in enumerate(stream, start=1):
        images = images.to(params['device'], non_blocking=True)
        target = target.to(params['device'], non_blocking=True).float().view(-1, 1)
        output = model(images)
        loss = criterion(output, target)
        roc_score = usr_roc_score(output, target)
        metric_monitor.update('Loss', loss.item())
        metric_monitor.update('ROC', roc_score)
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        optimizer.zero_grad()
        stream.set_description(
            "Epoch: {epoch}. Train.      {metric_monitor}".format(
                epoch=epoch,
                metric_monitor=metric_monitor)
        )

In [ ]:
def validate(val_loader, model, criterion, epoch, params):
    metric_monitor = MetricMonitor()
    model.eval()
    stream = tqdm(val_loader)
    final_targets = []
    final_outputs = []
    with torch.no_grad():
        for i, (images, target) in enumerate(stream, start=1):
            images = images.to(params['device'], non_blocking=True)
            target = target.to(params['device'], non_blocking=True).float().view(-1, 1)
            output = model(images)
            loss = criterion(output, target)
            roc_score = usr_roc_score(output, target)
            metric_monitor.update('Loss', loss.item())
            metric_monitor.update('ROC', roc_score)
            stream.set_description(
                "Epoch: {epoch}. Validation. {metric_monitor}".format(
                    epoch=epoch,
                    metric_monitor=metric_monitor)
            )
            
            targets = target.detach().cpu().numpy().tolist()
            outputs = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(outputs)
    return final_outputs, final_targets

In [ ]:
best_roc = -np.inf
best_epoch = -np.inf
best_model_name = None
for epoch in range(1, params['epochs'] + 1):
    train(train_loader, model, criterion, optimizer, epoch, params)
    predictions, valid_targets = validate(val_loader, model, criterion, epoch, params)
    roc_auc = round(roc_auc_score(valid_targets, predictions), 3)
    torch.save(model.state_dict(),f"{params['model']}_{epoch}_epoch_{roc_auc}_roc_auc.pth")
    if roc_auc > best_roc:
        best_roc = roc_auc
        best_epoch = epoch
        best_model_name = f"{params['model']}_{epoch}_epoch_{roc_auc}_roc_auc.pth"
    scheduler.step()

In [ ]:
print(f'The best ROC: {best_roc} was achieved on epoch: {best_epoch}.')
print(f'The Best saved model is: {best_model_name}')

# Prediction

In [ ]:
NUM_TTA = 1

Loading the best model from training step...

In [ ]:
model = AlienNet()
model.load_state_dict(torch.load(best_model_name))
model = model.to(params['device'])

In [ ]:
model.eval()
predicted_labels = None
for i in range(NUM_TTA):
    test_dataset = SETIDataset(
        images_filepaths = test_df['image_path'].values,
        targets = test_df['target'].values,
        transform = get_test_transforms(NUM_TTA)
    )
    test_loader = DataLoader(
        test_dataset, batch_size=params['batch_size'],
        shuffle=False, num_workers=params['num_workers'],
        pin_memory=True
    )
    
    temp_preds = None
    with torch.no_grad():
        for (images, target) in tqdm(test_loader):
            images = images.to(params['device'], non_blocking=True)
            output = model(images)
            predictions = torch.sigmoid(output).cpu().numpy()
            if temp_preds is None:
                temp_preds = predictions
            else:
                temp_preds = np.vstack((temp_preds, predictions))
    
    if predicted_labels is None:
        predicted_labels = temp_preds
    else:
        predicted_labels += temp_preds
        
predicted_labels /= NUM_TTA

In [ ]:
sub_df = pd.DataFrame()
sub_df['id'] = test_df['id']
sub_df['target'] = predicted_labels

In [ ]:
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv', index=False)

## Final Save Model

In [ ]:
torch.save(model.state_dict(), f"{params['model']}_{best_epoch}epochs_weights.pth")

This is a simple starter kernel on implementation of Transfer Learning using Pytorch for this problem. Pytorch has many SOTA Image models which you can try out using the guidelines in this notebook.

I hope you have learnt something from this notebook. I have created this notebook as a baseline model, which you can easily fork and paly-around with to get much better results. I might update parts of it down the line when I get more GPU hours and some interesting ideas.

**If you liked this notebook and use parts of it in you code, please show some support by upvoting this kernel. It keeps me inspired to come-up with such starter kernels and share it with the community.**

Thanks and happy kaggling!